## Functions

### Parse Frames & Sprites Functions

In [103]:
# Returns Parser with Frame and Sprite objects

import struct
from dataclasses import dataclass
from typing import List, Optional, Tuple
import os

@dataclass
class Sprite:
    y_pos: int
    x_pos: int
    size_tab: int
    tile_id: int
    palette_id: int
    hex_data: bytes
    
    # Calculated properties
    size_index: int
    tile_count: int
    dimensions: Tuple[int, int]
    starting_tile_offset: int
    width: int  # Added: width in tiles
    height: int  # Added: height in tiles
    pixel_width: int  # Added: width in pixels
    pixel_height: int  # Added: height in pixels
    priority: int  # Added: priority flag
    vflip: int  # Added: vertical flip flag
    hflip: int  # Added: horizontal flip flag

@dataclass
class Frame:
    sprstratt: int
    sprstrhot: int
    x_hot: int
    y_hot: int
    num_sprites: int
    sprites: List[Sprite]
    hex_data: bytes
    
class SpritesAnimParser:
    # Sprite size lookup table
    SIZE_TABLE = {
        0x0: (1, 1, 1),   # tiles, width, height
        0x1: (2, 1, 2),
        0x2: (3, 1, 3),
        0x3: (4, 1, 4),
        0x4: (2, 2, 1),
        0x5: (4, 2, 2),
        0x6: (6, 2, 3),
        0x7: (8, 2, 4),
        0x8: (3, 3, 1),
        0x9: (6, 3, 2),
        0xA: (9, 3, 3),
        0xB: (12, 3, 4),
        0xC: (4, 4, 1),
        0xD: (8, 4, 2),
        0xE: (12, 4, 3),
        0xF: (16, 4, 4),
    }
    
    # Tile size constants (8x8 pixels per tile)
    TILE_WIDTH = 8
    TILE_HEIGHT = 8
    
    def __init__(self, filename):
        self.filename = filename
        with open(filename, 'rb') as f:
            self.data = f.read()
        
        self.parse()
    
    def read_word(self, offset):
        """Read a 2-byte word (big-endian)"""
        return struct.unpack('>H', self.data[offset:offset+2])[0]
    
    def read_signed_word(self, offset):
        """Read a 2-byte signed word (big-endian)"""
        return struct.unpack('>h', self.data[offset:offset+2])[0]
    
    def parse(self):
        # Parse header
        signature = self.data[0:2]
        if signature != b'AA':
            if self.data[0:2].decode('ascii', errors='ignore') != 'AA':
                raise ValueError(f"Invalid file signature: {signature.hex()}")
        
        self.num_frames = self.read_word(0x02) + 1  # 0-based, so add 1
        
        # Check for frame signature in header
        if self.data[0x06:0x08] != b'SS' and self.data[0x06:0x08].decode('ascii', errors='ignore') != 'SS':
            raise ValueError(f"Expected frame signature 'SS' at offset 0x06")
        
        # Parse frames
        self.frames = []
        offset = 0x06  # First frame starts at 0x06
        
        # First pass: parse all frames
        for i in range(self.num_frames):
            frame, offset = self.parse_frame(offset)
            self.frames.append(frame)
        
        # Check for end of frames marker
        if self.data[offset:offset+2] != b'CC' and self.data[offset:offset+2].decode('ascii', errors='ignore') != 'CC':
            raise ValueError(f"Expected end of frames marker 'CC' at offset {offset:x}")
        
        # Mark the position of the last frame signature (CC)
        last_frame_signature_offset = offset
        
        # Tileset Data starts 2 bytes after the last frame signature
        tileset_data_start_offset = last_frame_signature_offset + 2
        
        # Read the tileset data size (first two bytes after CC signature)
        tileset_data_size = self.read_word(tileset_data_start_offset)
        
        # The actual tileset data starts after the size value
        self.tileset_data_offset = tileset_data_start_offset + 2
        
        # Palette Data Start = first two bytes after last frame signature + Tileset Data offset
        self.palette_data_offset = tileset_data_start_offset + tileset_data_size
        
        # Second pass: update sprite tile offsets now that we know the tileset data offset
        for frame in self.frames:
            for sprite in frame.sprites:
                # Recalculate starting tile offset with the tileset data offset
                tile_offset = ((sprite.tile_id & 0x07FF) | ((sprite.size_tab & 0xF000) >> 1)) * 32
                sprite.starting_tile_offset = self.tileset_data_offset + tile_offset
    
    def parse_frame(self, offset):
        """Parse a single frame starting at the given offset"""
        frame_start = offset
        
        # Check frame signature
        frame_sig = self.data[offset:offset+2]
        if frame_sig != b'SS' and frame_sig.decode('ascii', errors='ignore') != 'SS':
            raise ValueError(f"Invalid frame signature at offset {offset:x}: {frame_sig.hex()}")
        
        # Read frame header values
        sprstratt = self.read_word(offset + 0x0A)
        sprstrhot = self.read_word(offset + 0x0C)
        x_hot = self.read_signed_word(offset + 0x18)
        y_hot = self.read_signed_word(offset + 0x1A)
        num_sprites = self.read_word(offset + 0x24) + 1  # 0-based
        
        # Parse sprites
        sprites = []
        sprite_offset = offset + 0x26
        
        for _ in range(num_sprites):
            sprite, sprite_offset = self.parse_sprite(sprite_offset)
            sprites.append(sprite)
        
        frame_end = sprite_offset
        frame_hex_data = self.data[frame_start:frame_end]
        
        frame = Frame(
            sprstratt=sprstratt,
            sprstrhot=sprstrhot,
            x_hot=x_hot,
            y_hot=y_hot,
            num_sprites=num_sprites,
            sprites=sprites,
            hex_data=frame_hex_data
        )
        
        return frame, frame_end
    
    def parse_sprite(self, offset):
        """Parse a single sprite starting at the given offset"""
        sprite_start = offset
        
        y_pos = self.read_signed_word(offset)
        size_tab = self.read_word(offset + 2)
        tile_id = self.read_word(offset + 4)
        x_pos = self.read_signed_word(offset + 6)
        
        # Extract size index
        size_index = (size_tab >> 8) & 0x0F
        
        # Look up sprite dimensions and tile count
        if size_index in self.SIZE_TABLE:
            tile_count, width, height = self.SIZE_TABLE[size_index]
            dimensions = (width, height)
        else:
            tile_count = 0
            width = 0
            height = 0
            dimensions = (0, 0)
        
        # Calculate palette ID (bit 14 of tile_id)
        palette_id = (tile_id >> 13) & 0x03
        
        # Calculate starting tile offset (will be updated after we know tileset data offset)
        tile_offset = ((tile_id & 0x07FF) | ((size_tab & 0xF000) >> 1)) * 32

         # Extract priority, vflip, and hflip from tile_id
        priority = (tile_id >> 12) & 1
        vflip = (tile_id >> 12) & 1
        hflip = (tile_id >> 13) & 1
        
        sprite_end = offset + 8
        sprite_hex_data = self.data[sprite_start:sprite_end]
        
        # Calculate pixel dimensions
        pixel_width = width * self.TILE_WIDTH
        pixel_height = height * self.TILE_HEIGHT
        
        sprite = Sprite(
            y_pos=y_pos,
            x_pos=x_pos,
            size_tab=size_tab,
            tile_id=tile_id,
            palette_id=palette_id,
            hex_data=sprite_hex_data,
            size_index=size_index,
            tile_count=tile_count,
            dimensions=dimensions,
            starting_tile_offset=tile_offset,  # This will be updated later
            width=width,
            height=height,
            pixel_width=pixel_width,
            pixel_height=pixel_height,
            priority=priority,  # Added
            vflip=vflip,        # Added
            hflip=hflip         # Added
        )
        
        return sprite, sprite_end
    
    def print_analysis(self):
        """Print the analysis of the parsed file"""
        print(f"Analysis of: {os.path.basename(self.filename)}")
        print("=" * 50)
        print(f"Total number of frames: {self.num_frames}")
        print(f"Tileset Data offset: 0x{self.tileset_data_offset:04X}")
        print(f"Palette Data offset: 0x{self.palette_data_offset:04X}")
        print()
        
        for i, frame in enumerate(self.frames):
            print(f"Frame {i}:")
            print(f"  SprStrAtt: 0x{frame.sprstratt:04X}")
            print(f"  SprStrHot: 0x{frame.sprstrhot:04X}")
            print(f"  X Hot: {frame.x_hot}")
            print(f"  Y Hot: {frame.y_hot}")
            print(f"  Number of sprites: {frame.num_sprites}")
            print(f"  Frame Hex Data: {frame.hex_data.hex().upper()}")
            print()
            
            for j, sprite in enumerate(frame.sprites):
                print(f"  Sprite {j}:")
                print(f"    Size: {sprite.width}x{sprite.height} tiles ({sprite.pixel_width}x{sprite.pixel_height} pixels)")
                print(f"    Number of tiles: {sprite.tile_count}")
                print(f"    Starting tile offset: 0x{sprite.starting_tile_offset:04X}")
                print(f"    X Position: {sprite.x_pos}")
                print(f"    Y Position: {sprite.y_pos}")
                print(f"    Tile ID: 0x{sprite.tile_id:04X}")
                print(f"    Palette ID: {sprite.palette_id}")
                print(f"    Priority: {sprite.priority}")  # Added
                print(f"    Vertical Flip (vflip): {sprite.vflip}")  # Added
                print(f"    Horizontal Flip (hflip): {sprite.hflip}")  # Added
                print(f"    Sprite Hex Data: {sprite.hex_data.hex().upper()}")
                print()
            print("-" * 40)
            print()

# Helper function to debug file structure
def debug_file_structure(filename, max_bytes=256):
    """Debug function to examine the file structure"""
    with open(filename, 'rb') as f:
        data = f.read(max_bytes)
    
    print(f"First {max_bytes} bytes of {os.path.basename(filename)}:")
    print("Offset | Hex Values | ASCII")
    print("-" * 50)
    
    for i in range(0, min(len(data), max_bytes), 16):
        hex_values = ' '.join([f'{b:02X}' for b in data[i:i+16]])
        ascii_values = ''.join([chr(b) if 32 <= b <= 126 else '.' for b in data[i:i+16]])
        print(f'{i:06X} | {hex_values:<48} | {ascii_values}')

# Main parsing function
def analyze_sprites_file(filename):
    """
    Analyze a sprites.anim file and display the results
    
    Args:
        filename: Path to the sprites.anim file
    """
    try:
        # First, let's examine the file structure
        debug_file_structure(filename, 128)
        print("\n" + "="*50 + "\n")
        
        # Now parse the file
        parser = SpritesAnimParser(filename)
        parser.print_analysis()
        return parser
    except Exception as e:
        print(f"Error analyzing file: {e}")
        import traceback
        traceback.print_exc()
        return None

### Decode Genesis Tiles as PNG Functions

In [104]:
# Retruns an Image
# Required input parameters:
# - tile_hex_string: Hexadecimal string representing the sprite's tile data.
# - tiles_wide: Number of tiles horizontally in the sprite.
# - tiles_high: Number of tiles vertically in the sprite.
# - palette_hex_string: Hexadecimal string representing the color palette.

import binascii
from PIL import Image
import io  # Needed for displaying in some notebook environments

def decode_genesis_sprite(tile_hex_string, tiles_wide, tiles_high, palette_hex_string):
    """
    Decodes Sega Genesis 4bpp linear tile data into a Pillow Image object.

    Assumes tiles in the hex string are stored in column-major order.

    Args:
        tile_hex_string (str): A string containing space-separated hex bytes
                               representing the 4bpp tile data.
        tiles_wide (int): The number of tiles horizontally in the sprite.
        tiles_high (int): The number of tiles vertically in the sprite.
        palette_hex_string (str): A string containing space-separated 6-digit
                                  RRGGBB hex color values for the 16-color palette.

    Returns:
        PIL.Image.Image: A Pillow Image object in 'P' (paletted) mode,
                         or None if a critical error occurred.

    Raises:
        ValueError: If tile_hex_string contains invalid hex data or if the
                    amount of data doesn't match the expected size.
    """
    # --- Configuration (Constants for Genesis 8x8 4bpp tiles) ---
    TILE_WIDTH_PX = 8
    TILE_HEIGHT_PX = 8
    BPP = 4 # Bits per pixel

    # 1. Parse Palette
    palette_colors_hex = palette_hex_string.split()
    if len(palette_colors_hex) < 16:
        print(f"Warning: Provided palette has {len(palette_colors_hex)} colors, expected 16. Padding with black.")
        palette_colors_hex.extend(["000000"] * (16 - len(palette_colors_hex)))

    # Convert hex palette to list of RGB tuples using the helper
    palette_rgb = [_hex_rgb_to_tuple(hex_col) for hex_col in palette_colors_hex[:16]] # Take only first 16

    # Flatten palette for Pillow (R1, G1, B1, R2, G2, B2, ...)
    flat_palette = [component for color in palette_rgb for component in color]
    # Pad the Pillow palette to 768 bytes (256 colors * 3 channels)
    flat_palette.extend([0] * (768 - len(flat_palette)))

    # 2. Parse Tile Data
    # Remove whitespace and newlines before converting
    tile_data_hex_clean = "".join(tile_hex_string.split())
    try:
        tile_data_bytes = binascii.unhexlify(tile_data_hex_clean)
    except binascii.Error as e:
        raise ValueError(f"Error converting tile hex data to bytes: {e}. Invalid hex data.") from e

    # Verify data size
    bytes_per_tile = (TILE_WIDTH_PX * TILE_HEIGHT_PX * BPP) // 8
    num_tiles = tiles_wide * tiles_high
    expected_bytes = num_tiles * bytes_per_tile

    if len(tile_data_bytes) < expected_bytes:
         raise ValueError(f"Input data is too short! Expected {expected_bytes} bytes for a {tiles_wide}x{tiles_high} sprite, got {len(tile_data_bytes)} bytes.")
    elif len(tile_data_bytes) > expected_bytes:
         print(f"Warning: Input data is longer than expected ({len(tile_data_bytes)} vs {expected_bytes}). Using first {expected_bytes} bytes.")
         tile_data_bytes = tile_data_bytes[:expected_bytes]


    # 3. Create Image and Decode Pixels (Assuming Column-Major Order in Source Data)
    img_width = tiles_wide * TILE_WIDTH_PX
    img_height = tiles_high * TILE_HEIGHT_PX
    img = Image.new('P', (img_width, img_height))
    img.putpalette(flat_palette)

    # Iterate through the *output grid positions* (column by column)
    for tile_col in range(tiles_wide):
        for tile_row in range(tiles_high):
            # Calculate the index in the *linear source data* based on column-major assumption
            linear_tile_index = tile_col * tiles_high + tile_row

            # This check should be redundant due to the size check above, but safe to keep
            if linear_tile_index >= num_tiles:
                print(f"Error: Calculated linear tile index {linear_tile_index} is out of bounds (max {num_tiles-1}). Skipping tile.")
                continue

            tile_start_byte = linear_tile_index * bytes_per_tile

            # --- Decode the pixels for this specific tile ---
            tile_pixels = [] # Palette indices for this 8x8 tile
            for i in range(bytes_per_tile):
                byte_offset = tile_start_byte + i
                byte_val = tile_data_bytes[byte_offset]
                pixel1_index = (byte_val >> 4) & 0x0F # High nibble
                pixel2_index = byte_val & 0x0F        # Low nibble
                tile_pixels.append(pixel1_index)
                tile_pixels.append(pixel2_index)
            # --- Finished decoding pixels for this tile ---

            # --- Place these tile pixels onto the main image canvas ---
            start_x = tile_col * TILE_WIDTH_PX
            start_y = tile_row * TILE_HEIGHT_PX

            for y_in_tile in range(TILE_HEIGHT_PX):
                for x_in_tile in range(TILE_WIDTH_PX):
                    pixel_index_in_tile = y_in_tile * TILE_WIDTH_PX + x_in_tile
                    palette_idx = tile_pixels[pixel_index_in_tile]

                    # Check index validity against palette size (should be 0-15)
                    if palette_idx >= len(palette_rgb): # len(palette_rgb) is 16
                        print(f"Warning: Pixel data uses palette index {palette_idx}, which is invalid for a 16-color palette. Using index 0.")
                        palette_idx = 0

                    img.putpixel((start_x + x_in_tile, start_y + y_in_tile), palette_idx)

    return img

def _hex_rgb_to_tuple(hex_rgb):
    """Converts a 6-digit hex RGB string (e.g., '90FCFC') to an RGB tuple (0-255)."""
    hex_rgb = hex_rgb.lstrip('#') # Remove leading # if present
    if len(hex_rgb) != 6:
        # Use warning instead of print for library-like function
        import warnings
        warnings.warn(f"Invalid hex color length '{hex_rgb}'. Using black (0,0,0).")
        return (0, 0, 0)
    try:
        r = int(hex_rgb[0:2], 16)
        g = int(hex_rgb[2:4], 16)
        b = int(hex_rgb[4:6], 16)
        return (r, g, b)
    except ValueError:
        import warnings
        warnings.warn(f"Invalid hex color value '{hex_rgb}'. Using black (0,0,0).")
        return (0, 0, 0)


### Export Sprites To PNG Function

In [105]:
# Function to export sprites to PNG files
# Required input parameters:
# - parser: SpritesAnimParser object containing parsed sprite data
# - output_dir: Directory where PNG files will be saved (default: 'frames_output')
# - palette_hex: Hex string representing the color palette (default provided)

import os

#90FCFC FCFCFC D8FCFC D8D8D8 000000 B4B4B4 6C6C6C 484848 ECC8C8 FC9090 D8486C 486CD8 D8B424 6C90FC 242424 FCFCFC 
#90FCFC D8D8D8 909090 484848 242424 D89000 B46C00 000000 242490 00006C 244800 002400 B44800 6C0000 B46C6C 6C4824 
#90FCFC 242424 000000 000000 242424 242424 D89090 B46C6C D84800 FCB400 FCFCFC D8D8D8 FCFCFC B4B4B4 D89048 4890B4 
#90FCFC 242424 000000 242400 244824 246C48 D89090 B46C6C D86C00 FC9024 D8D8D8 249024 24B424 246C24 D89048 4890B4 

def export_sprites_to_png(parser, output_dir='frames_output', palette_hex="90FCFC 242424 000000 000000 242424 242424 D89090 B46C6C D84800 FCB400 FCFCFC D8D8D8 FCFCFC B4B4B4 D89048 4890B4"):
    """
    Export all sprites from parsed data to PNG files using the existing decode_genesis_sprite function.
    
    Args:
        parser: SpritesAnimParser object with parsed data
        output_dir: Base directory for output
        palette_hex: Hex string of palette colors
    """
    # Create base output directory
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Process each frame
    for frame_idx, frame in enumerate(parser.frames):
        # Create frame directory
        frame_dir = os.path.join(output_dir, f'frame_{frame_idx}')
        if not os.path.exists(frame_dir):
            os.makedirs(frame_dir)
        
        #print(f"Processing frame {frame_idx}...")
        
        # Process each sprite in the frame
        for sprite_idx, sprite in enumerate(frame.sprites):
            # Extract tile data for this sprite
            tile_data = []
            for tile_idx in range(sprite.tile_count):
                tile_offset = sprite.starting_tile_offset + (tile_idx * 32)
                tile_bytes = parser.data[tile_offset:tile_offset + 32]
                tile_data.extend(tile_bytes)
            
            # Convert to hex string
            tile_hex_string = ''.join([f'{b:02X}' for b in tile_data])
            
            # Call your existing function to create sprite image
            sprite_image = decode_genesis_sprite(
                tile_hex_string,
                sprite.width,
                sprite.height,
                palette_hex
            )
            
            # Save sprite image
            filename = f'sprite_{sprite_idx}.png'
            filepath = os.path.join(frame_dir, filename)
            sprite_image.save(filepath)
            
            #print(f"  Saved sprite {sprite_idx} to {filepath}")
    
    #print(f"\nExport complete! All sprites saved to {output_dir}")


### Combine Frame Sprites As 1 PNG Function

In [106]:
# Functions to create a frame image from multiple sprites
from PIL import Image
import os
import math

# --- Constants ---
TILE_SIZE = 8  # Standard Sega Genesis tile size is 8x8 pixels

# --- Helper Function ---
def calculate_sprite_bounds(sprite_info):
    """Calculates sprite pixel dimensions and bounding box relative to the hotspot."""
    width_px = sprite_info["size_tiles"][0] * TILE_SIZE
    height_px = sprite_info["size_tiles"][1] * TILE_SIZE

    # Top-left corner relative to hotspot
    top_left_x = sprite_info["pos_x"]
    top_left_y = sprite_info["pos_y"]

    # Bottom-right corner relative to hotspot
    bottom_right_x = top_left_x + width_px
    bottom_right_y = top_left_y + height_px

    return {
        "width_px": width_px,
        "height_px": height_px,
        "top_left_x": top_left_x,
        "top_left_y": top_left_y,
        "bottom_right_x": bottom_right_x,
        "bottom_right_y": bottom_right_y,
    }

# --- Core Frame Creation Function ---
def create_frame_image(frame_data, sprite_base_path, sprite_file_template, output_filename):
    """
    Combines individual sprite PNGs into a single frame image based on frame data.

    Args:
        frame_data (dict): Dictionary containing data for a single frame.
                           Expected keys: 'frame_index', 'hotspot_x', 'hotspot_y',
                           'sprites' (list of sprite info dicts).
                           Each sprite info dict needs: 'index', 'size_tiles', 'pos_x', 'pos_y'.
        sprite_base_path (str): The directory where the individual sprite PNGs are located.
        sprite_file_template (str): A format string for sprite filenames,
                                    e.g., "sprite_{index}.png".
        output_filename (str): The full path and filename for the final combined image.

    Returns:
        bool: True if the frame image was created successfully, False otherwise.
    """
    frame_index = frame_data.get("frame_index", "Unknown")
    print(f"\n--- Processing Frame {frame_index} ---")

    if not frame_data.get("sprites"):
        print(f"  No sprites defined for frame {frame_index}. Skipping.")
        # Optionally create an empty or 1x1 transparent image if needed
        # Example: Image.new('RGBA', (1, 1), (0, 0, 0, 0)).save(output_filename)
        return True  # Consider it success if there's nothing to do

    if len(frame_data["sprites"]) == 1:
        print(f"  Only one sprite defined for frame {frame_index}. Skipping.")
        return True  # Exit if there's only one sprite

    # 1. Calculate overall bounding box to determine canvas size
    min_x = float('inf')
    min_y = float('inf')
    max_x = float('-inf')
    max_y = float('-inf')

    sprite_details = [] # Store calculated details and image objects
    has_errors = False

    print("  Loading and analyzing sprites...")
    for sprite_info in frame_data["sprites"]:
        sprite_index = sprite_info["index"]
        sprite_filename_only = sprite_file_template.format(index=sprite_index)
        sprite_full_path = os.path.join(sprite_base_path, sprite_filename_only)

        if not os.path.exists(sprite_full_path):
            print(f"  Error: Sprite file not found: {sprite_full_path}")
            has_errors = True
            continue # Skip this sprite, maybe continue with others? Or return False immediately?

        try:
            img = Image.open(sprite_full_path).convert("RGBA") # Ensure RGBA for transparency
        except Exception as e:
            print(f"  Error opening or converting sprite {sprite_full_path}: {e}")
            has_errors = True
            continue

        bounds = calculate_sprite_bounds(sprite_info)

        # Check if loaded image size matches expected size
        if img.width != bounds["width_px"] or img.height != bounds["height_px"]:
            print(f"  Warning: Sprite {sprite_filename_only} dimensions ({img.width}x{img.height}) "
                  f"do not match expected size ({bounds['width_px']}x{bounds['height_px']}) "
                  f"based on tile data.")

        sprite_details.append({
            "info": sprite_info,
            "bounds": bounds,
            "image": img
        })

        # Update overall min/max coordinates (relative to hotspot)
        min_x = min(min_x, bounds["top_left_x"])
        min_y = min(min_y, bounds["top_left_y"])
        max_x = max(max_x, bounds["bottom_right_x"])
        max_y = max(max_y, bounds["bottom_right_y"])

        # print(f"    Sprite {sprite_index} ({sprite_filename_only}): "
        #       f"Size={bounds['width_px']}x{bounds['height_px']}, "
        #       f"PosRelHotspot=({sprite_info['pos_x']}, {sprite_info['pos_y']}), "
        #       f"BoundsRelHotspot=[{bounds['top_left_x']}:{bounds['bottom_right_x']}, {bounds['top_left_y']}:{bounds['bottom_right_y']}]")

    if has_errors:
        print(f"  Errors encountered while loading sprites for frame {frame_index}. Frame not created.")
        return False

    if not sprite_details: # Should only happen if errors occurred and we continued
         print(f"  No valid sprites found to process for frame {frame_index}.")
         return False

    # 2. Determine canvas size and the offset for pasting
    # Handle cases where max == min (e.g., single point) by ensuring minimum 1 pixel size
    canvas_width = max(1, max_x - min_x)
    canvas_height = max(1, max_y - min_y)

    offset_x = -min_x
    offset_y = -min_y

    print(f"  Overall Bounds Rel Hotspot: X=[{min_x}, {max_x}], Y=[{min_y}, {max_y}]")
    print(f"  Calculated Canvas Size: {canvas_width} x {canvas_height}")
    print(f"  Placement Offset (Hotspot's pos on canvas): ({offset_x}, {offset_y})")

    # 3. Create the canvas
    final_image = Image.new('RGBA', (canvas_width, canvas_height), (0, 0, 0, 0))

    # 4. Paste sprites onto the canvas
    print(f"  Pasting {len(sprite_details)} sprites onto canvas...")
    for details in sprite_details:
        sprite_img = details["image"]
        sprite_info = details["info"]

        paste_x = sprite_info["pos_x"] + offset_x
        paste_y = sprite_info["pos_y"] + offset_y

        # print(f"    Pasting sprite {details['info']['index']} at canvas coordinates: ({paste_x}, {paste_y})")
        final_image.paste(sprite_img, (paste_x, paste_y), sprite_img)

    # 5. Save the final image
    try:
        # Ensure output directory exists
        output_dir = os.path.dirname(output_filename)
        if output_dir: # Only create if path contains a directory part
            os.makedirs(output_dir, exist_ok=True)

        final_image.save(output_filename)
        print(f"  Successfully combined sprites into {output_filename}")
        # Optional: Print hotspot info relative to canvas
        # frame_hotspot_x = frame_data['hotspot_x']
        # frame_hotspot_y = frame_data['hotspot_y']
        # print(f"  Frame Hotspot ({frame_hotspot_x}, {frame_hotspot_y}) corresponds to canvas coordinates: ({offset_x}, {offset_y})")

        return True
    except Exception as e:
        print(f"  Error saving final image {output_filename}: {e}")
        return False

## Call Functions

In [107]:
# This calls all the necessary functions to parse the file, create the frames, sprites, and PNGs

# Populates parser object ...
parser = analyze_sprites_file('../src/graphics/sprites.anim')

# Export sprites to PNG files
export_sprites_to_png(parser)

# Define paths and naming template
sprite_base_path = "frames_output"  # Base path where individual sprite PNGs are located
sprite_file_template = "frame_{frame_idx}/sprite_{index}.png"
output_filename_template = "frames_output/frame_{frame_idx}/frame_{frame_idx}_combined.png"

# Iterate through frames and create combined images
for frame_idx, frame in enumerate(parser.frames):
    # Prepare frame data for create_frame_image
    frame_data = {
        "frame_index": frame_idx,
        "hotspot_x": frame.x_hot,
        "hotspot_y": frame.y_hot,
        "sprites": [
            {
                "index": sprite_idx,
                "size_tiles": (sprite.width, sprite.height),
                "pos_x": sprite.x_pos,
                "pos_y": sprite.y_pos
            }
            for sprite_idx, sprite in enumerate(frame.sprites)
        ]
    }

    # Format the output filename
    output_filename = output_filename_template.format(frame_idx=frame_idx)    

    # Call create_frame_image
    create_frame_image(
        frame_data,
        sprite_base_path,
        sprite_file_template.format(frame_idx=frame_idx, index="{index}"),  # Pass formatted template
        output_filename
    )     

First 128 bytes of sprites.anim:
Offset | Hex Values | ASCII
--------------------------------------------------
000000 | 41 41 02 24 00 02 53 53 00 CD 00 C2 00 20 00 40  | AA.$..SS..... .@
000010 | 00 00 00 00 00 00 00 FF 00 00 00 00 00 FF FF FC  | ................
000020 | FF EC 00 00 00 00 00 00 00 FF 00 00 FF E7 07 00  | ................
000030 | 46 A0 FF F9 53 53 00 CD 00 C2 00 30 00 40 00 00  | F...SS.....0.@..
000040 | 00 00 00 00 00 FF 00 00 00 00 00 FF FF FF FF E8  | ................
000050 | 00 00 00 00 00 00 00 FF 00 01 FF E5 07 00 46 98  | ..............F.
000060 | FF F9 FF FD 20 00 46 0D FF F1 53 53 00 CD 00 C3  | .... .F...SS....
000070 | 00 20 00 40 00 00 00 00 00 00 00 FF 00 00 00 00  | . .@............


Analysis of: sprites.anim
Total number of frames: 549
Tileset Data offset: 0x8928
Palette Data offset: 0x9F34

Frame 0:
  SprStrAtt: 0x0000
  SprStrHot: 0x0000
  X Hot: -4
  Y Hot: -20
  Number of sprites: 1
  Frame Hex Data: 535300CD00C20020004000000000000000FF00000000

## Create Animation from PNG

In [108]:
import os
import glob
from PIL import Image
import numpy as np
import imageio

def create_animation_gif(start_frame, end_frame, direction_num, output_filename, animation_data=None):
    """
    Create an animated GIF for a specific direction of animation frames
    
    Parameters:
    - start_frame: starting frame number
    - end_frame: ending frame number
    - direction_num: 0-7 for the direction
    - output_filename: name of output GIF file
    - animation_data: list of frame durations from frames.asm
    """
    # Calculate frame numbers for the specified direction
    frame_offset = direction_num * 5  # 5 frames per direction
    frame_numbers = [start_frame + frame_offset + i for i in range(5)]
    
    # Default frame durations if not provided (60fps game = ~16.7ms per frame)
    if animation_data is None:
        durations = [167] * len(frame_numbers)  # 167ms = ~6fps
    else:
        # Convert game frames to milliseconds (assuming 60fps)
        durations = [abs(frame_time) * 1000 / 60 for frame_time in animation_data]
    
    # Collect all combined frames
    frames = []
    max_width = 0
    max_height = 0
    
    # First pass to find the maximum dimensions
    temp_frames = []
    for frame_num in frame_numbers:
        frame_dir = f"frames_output/frame_{frame_num}"
        
        # Check if there's a combined image
        combined_path = f"{frame_dir}/frame_{frame_num}_combined.png"
        
        if os.path.exists(combined_path):
            # Use the pre-combined image
            img = Image.open(combined_path)
            width, height = img.size
            temp_frames.append(img)
        else:
            # Find all sprite PNGs for this frame
            sprite_paths = sorted(glob.glob(f"{frame_dir}/sprite_*.png"))
            
            if not sprite_paths:
                print(f"Warning: No sprites found for frame {frame_num}")
                continue
                
            # Open all sprites to calculate total dimensions
            sprites = [Image.open(path) for path in sprite_paths]
            
            # Create a transparent base image
            # For simplicity, just use dimensions of first sprite
            base_img = sprites[0]
            width, height = base_img.size
            
            # Create a new image with the sprites
            combined = Image.new("RGBA", (width, height), (0, 0, 0, 0))
            
            # Paste all sprites
            for sprite in sprites:
                combined.paste(sprite, (0, 0), sprite)
            
            temp_frames.append(combined)
            
        # Update maximum dimensions
        max_width = max(max_width, width)
        max_height = max(max_height, height)
    
    # Second pass to resize all frames to the same dimensions
    for img in temp_frames:
        # Create a new image with the maximum dimensions
        resized = Image.new("RGBA", (max_width, max_height), (0, 0, 0, 0))
        # Paste the original image centered
        offset_x = (max_width - img.width) // 2
        offset_y = (max_height - img.height) // 2
        resized.paste(img, (offset_x, offset_y), img)
        # Convert to RGB for GIF compatibility
        rgb_img = Image.new("RGB", resized.size, (0, 0, 0))
        rgb_img.paste(resized, (0, 0), resized)
        frames.append(np.array(rgb_img))
    
    # Create output directory if needed
    os.makedirs(os.path.dirname(output_filename), exist_ok=True)
    
    if frames:
        # Save as GIF
        imageio.mimsave(output_filename, frames, duration=durations, loop=0)
        print(f"Created animation: {output_filename}")
    else:
        print(f"Warning: No frames to save for {output_filename}")

def create_all_direction_gifs(animation_name, start_frame, animation_data=None, duration=5):
    """
    Create GIFs for all 8 directions of a specific animation
    
    Parameters:
    - animation_name: Name of the animation (for output filename)
    - start_frame: The starting frame number of this animation
    - animation_data: List of frame durations from frames.asm
    - duration: Number of frames per direction (default 5)
    """
    directions = ["up", "up_right", "right", "down_right", 
                 "down", "down_left", "left", "up_left"]
    
    # Create output directory
    output_dir = f"animations/{animation_name}"
    os.makedirs(output_dir, exist_ok=True)
    
    # For each direction, create a GIF
    for i, direction in enumerate(directions):
        output_file = f"{output_dir}/{animation_name}_{direction}.gif"
        create_animation_gif(
            start_frame=start_frame,
            end_frame=start_frame + (8 * duration) - 1,
            direction_num=i,
            output_filename=output_file,
            animation_data=animation_data
        )

# Example usage:
if __name__ == "__main__":
    # Animation timing data from frames.asm
    # SPAskatewp animation timings (see frames.asm line ~530)
    skatewp_timings = [10, 10, 10, 10, -15]  # Negative means loop back
    
    # SPAskate animation timings
    skate_timings = [10, 10, 10, 10, -15]

create_animation_gif(
    start_frame=0,
    end_frame=39,
    direction_num=2,  # 2 = right direction
    output_filename="animations/skating/skating_right.gif",
    animation_data=[10, 10, 10, 10, -15]  # Skating animation timing from frames.asm
)
   
    
   
   

Created animation: animations/skating/skating_right.gif


In [109]:
# Extract all unique Sprite Tile IDs and their corresponding size_tab values
unique_tile_data = set()

for frame in parser.frames:
    for sprite in frame.sprites:
        unique_tile_data.add((sprite.tile_id, sprite.size_tab))

# Sort the data by tile_id in descending order
sorted_tile_data = sorted(unique_tile_data, key=lambda x: x[0], reverse=True)

# Print the unique Tile IDs and size_tab values as hex
print(f"Unique Sprite Tile Data ({len(sorted_tile_data)}):")
for tile_id, size_tab in sorted_tile_data:
    print(f"Tile ID: 0x{tile_id:04X}, Size Tab: 0x{size_tab:04X}")

Unique Sprite Tile Data (1739):
Tile ID: 0x5EF5, Size Tab: 0x0000
Tile ID: 0x5CC0, Size Tab: 0x1000
Tile ID: 0x5BD6, Size Tab: 0x2000
Tile ID: 0x559F, Size Tab: 0x2000
Tile ID: 0x53D0, Size Tab: 0x2000
Tile ID: 0x50F4, Size Tab: 0x2000
Tile ID: 0x50F2, Size Tab: 0x2000
Tile ID: 0x5083, Size Tab: 0x2000
Tile ID: 0x4F1C, Size Tab: 0x1500
Tile ID: 0x4F18, Size Tab: 0x1500
Tile ID: 0x4F14, Size Tab: 0x1500
Tile ID: 0x4F10, Size Tab: 0x1500
Tile ID: 0x4F0C, Size Tab: 0x1500
Tile ID: 0x4EA8, Size Tab: 0x0300
Tile ID: 0x4E06, Size Tab: 0x1900
Tile ID: 0x4E00, Size Tab: 0x1900
Tile ID: 0x4DDA, Size Tab: 0x2000
Tile ID: 0x4DC9, Size Tab: 0x2000
Tile ID: 0x4DB6, Size Tab: 0x2000
Tile ID: 0x4D8B, Size Tab: 0x2000
Tile ID: 0x4D48, Size Tab: 0x1D00
Tile ID: 0x4D20, Size Tab: 0x2000
Tile ID: 0x4CF0, Size Tab: 0x2000
Tile ID: 0x4CA9, Size Tab: 0x2000
Tile ID: 0x4C47, Size Tab: 0x2000
Tile ID: 0x4C2F, Size Tab: 0x2000
Tile ID: 0x4C2D, Size Tab: 0x2000
Tile ID: 0x4C1E, Size Tab: 0x2000
Tile ID: 0x4C1D,